# Introduction to Q#: A high level quantum development language for everyone
### Seminar for [University of Technology Sydney](https://www.uts.edu.au/research-and-teaching/our-research/centre-quantum-software-and-information/events/online-seminar-dr-sarah-kaiser-wa-usa) 

Dr. Sarah Kaiser |  [@crazy4pi314](twitter.com/crazy4pi314) |  7 Sept 2020

---

Talk slides/Jupyter Notebook can be found at [bit.ly/qsharp-uts](TODO)
    
<center>
    <a src="http://unitary.fund"> <img src="https://img.shields.io/badge/Supported%20By-UNITARY%20FUND-brightgreen.svg?style=flat" width="20%" align="center"/> </a>
    <img src="https://img.shields.io/github/license/crazy4pi314/uts-qsharp-intro" width="8.5%" align="center"/>
</center>   

### Abstract
As the field of quantum computing expands from the academic to the industry realm, we need a way that we can continue to collaborate and innovate in both regimes. Open source quantum software development platforms like the Quantum Development Kit and Q# from Microsoft, serve as a bridge to connect research ideas to reality.
In this talk, I will give you a tour of what you can do with Q# and show you an example of how I am using it in my own research on qRAMs. After this talk, you will have the resources you need to dive into using Q# for your own research projects!

---

#### Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/quantum/install-guide/python?view=qsharp-preview) or you can run this presentation in your browser [here](TODO).

## whoami: author, streamer, community builder...
<br>
<center>
<img src="media/kaiser-bio.png" width="55%" align="center"/>
</center>

## ... and a researcher  👩‍💻

<br>
<center>
    <img src="media/kaiser-lab.jpg" width="35%">
</center>

## 📃Agenda
 
1. Give you a tour of Q# and why we are using it for our research, 
2. show you a case study of using Q# to do research on qRAM, and
3. share some open source tools and resources for how you can leverage Q# for your own research and studies!


# Part 1: A tour of Q#

## So many options...

https://qosf.org/project_list/

<center>
    <img src="media/qsof.gif" width="50%">

</center>

### What we need:

- We want to implement _algorithms_
  - We want to think about qRAM algorithms at a high level, not always specifically at the gate level
  - Need the flexibility to create gate level optimizations as well

- We want to build tools to enable research and collaboration
  - Need ways to packages our work and make it easy to share, collaborate, and reproduce
  - We want to work with a community where we are all included

# 📊 What do you know about Q#?

# Q\# : Microsoft's quantum programming language

- New open source language that is domain-specific for quantum computing
- Used with the [_Quantum Development Kit_](https://www.microsoft.com/en-us/quantum/development-kit) which provides lots of tools for writing and running your programs.
- Designed to be integrated with a number of languages/platforms like Python and .NET

<center>
    <img src="media/stack.png" width="80%">
</center>



## What do I write in a Q# program?

- **Functions**: `Sin`, `ln`, reversing arrays, etc.
    - Deterministic actions, similar to mathematical definition for functions
- **Operations**: Everything else 😁 
    - Working with qubits is always an operation

## Q# Hello World

- The `:` here shows you what type an variable or return of a function or operation is
- Everything in Q# is `()` in, `()` out

In [ ]:
function Greeting(name : String) : Unit {
    Message($"Hello World! Nice to meet you {name} 💖");
}

In [ ]:
%simulate Greeting name="Sarah"

## Q# Hello _Quantum_ world

In [ ]:
open Microsoft.Quantum.Measurement;

In [ ]:
operation Qrng() : Result {
    using (qubit = Qubit()) {   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
}

In [ ]:
%simulate Qrng

## Functions + Types in Q#

- Functions are ways to define deterministic, classical calculations.
- Q# is a strongly typed language, with a variety of built-in types and ways for you to define your own!

In [ ]:
// Library has the definition of the Complex type we use below.
open Microsoft.Quantum.Math;

In [ ]:
// A user defined type for an array of complex numbers.
newtype ComplexArray = (Count : Int, Values : Complex[]);

In [ ]:
// Converts an array of real numbers to a ComplexArray
function AsComplexArray(data : Double[]) : ComplexArray {

    // Start with a new ComplexArray to put your results in
    mutable results = ComplexArray(0, new Complex[0]);

    for (item in data) {
        // update-and-reassign statement
        set results w/= Values <- results::Values + [Complex(item, 0.)]; 
    }
    
    // Return the results with the Count named item filled out
    return results w/ Count <- Length(results::Values); 
}

function ConvertedData() : ComplexArray {
    let data = [1.0, 2.0, 3.0];
    return AsComplexArray(data);
}

In [ ]:
%simulate ConvertedData

## `using` Qubits in Q#

- Qubits are a resource that are requested from the runtime when you need them and returned when you are done.

In [ ]:
operation Qrng() : Result {
    using (qubit = Qubit()) {   // Preparing the qubit
        H(qubit);               // Do operation H
        return MResetZ(qubit);  // Measure and reset qubit
    }
}

## Generating new operations in Q# ##

The _functors_ `Adjoint` and `Controlled` allow you to generate new operations without changes to your code to implement those versions.

In [ ]:
operation ApplyMultiControlledNOT(control: Qubit[], target : Qubit) : Unit {
    Controlled X(control, target);
}

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation UseCtlFunctor() : Unit {
    using ((controls, target) = (Qubit[2], Qubit())) {
        // Preparing uniform superposition
        ApplyToEach(H, controls);
        // Apply our controlledNOT
        ApplyMultiControlledNOT(controls, target);
        // Print simulator diagnostics
        DumpMachine();
        ResetAll(controls + [target]);
    }  
}

### Q# programs can be run from:

- the [command line](https://docs.microsoft.com/en-us/quantum/user-guide/host-programs?tabs=tabid-python#q-from-the-command-prompt), if built as stand-alone applications
- [Python](https://docs.microsoft.com/quantum/quickstarts/install-python?tabs=tabid-conda) or [.NET](https://docs.microsoft.com/quantum/quickstarts/install-cs?tabs=tabid-cmdline%2Ctabid-csharp) language programs (C#, F#, etc.) for easy data processing and visualization
- [Jupyter notebooks](https://docs.microsoft.com/quantum/quickstarts/install-jupyter?tabs=tabid-conda) with either Q# or Python kernels


In [ ]:
%simulate UseCtlFunctor

## More than just simulation...

In [ ]:
%lsmagic

### Example target machine: Execution tracing!

In [ ]:
%trace Qrng

### Example target machine: Resource estimation!

In [ ]:
%estimate UseCtlFunctor

## Unit testing in Q#

- Great way to check that what we have typed matches paper results 😊

In [ ]:
open Microsoft.Quantum.Arrays;

operation AllocateQubitRegister(numQubits : Int) : Unit {

    Fact(numQubits > 0, "Expected a positive number.");
    
    using (register = Qubit[numQubits]) {
        ApplyToEach(AssertQubit(Zero, _), register);
    }
    Message("Test passed!");
}

In [ ]:
%simulate AllocateQubitRegister numQubits=5

## Unit testing in Q# cont.

- Helpful to test if qRAM optimizations still do the same thing
- Q# uses the _Choi–Jamiłkowski isomorphism_ to make an assertion of operation equivalence to one about preparing states
    <!--- We know that if you apply an operation and then it's adjoint, that should be an Identity operation
    - Make a channel that applies your operation under test, and then the adjoint of your reference operation.
    - If you start with a maximally entangled state, then apply the channel to one half of a maximally entangled state, then you can use state assertions to verify that you still have that same maximally entangled state.-->


In [ ]:
open Microsoft.Quantum.Diagnostics; 

operation ApplyCNOT(register : Qubit[])
: Unit is Adj + Ctl {
    CNOT(register[0], register[1]);
}

In [ ]:
operation ApplyCNOTTheOtherWay(register : Qubit[])
: Unit is Adj + Ctl {
    within {
        ApplyToEachCA(H, register);
    } 
    apply {
        CNOT(register[1], register[0]);
    }
}

operation CheckThatThisWorks() : Unit {
    AssertOperationsEqualReferenced(2, ApplyCNOT, ApplyCNOTTheOtherWay);
    Message("It works!");
}

In [ ]:
%trace CheckThatThisWorks --depth=1

## Our plan:

🧰 **Step 1:** Use functions and operations to implement the qRAM proposals

📚 **Step 2:** Use libraries, functors, and other features to reduce how much code we need to write

🧪 **Step 3:** Use testing features to make sure our implementations are correct

💰 **Step 4:** Profit! (also publish 📃)

# Part 2: A case study of Q# driven research: The qRAM library: the musical

## Quantum applications _might_ need memory

- We need ways to transfer **classical data** to a **quantum system**
- _Some_ quantum algorithms, particularly quantum machine learning, assume access to a quantum RAM to load and store data during calculations.
- Queries at address $x$ can take many forms: 
    - Bit value as a phase (Grover's Algorithm): $\left|x\right\rangle\mapsto(-1)^{b_x}\left|x\right\rangle$
    - Bit value as a qubit (Element distinctness): $\left|x\right\rangle\left|0\right\rangle\mapsto\left|x\right\rangle\left|b_x\right\rangle$
    - Bit value as a complex vector of amplitudes (HHL Algorithm): $(b_0...b_n)\mapsto\sum\limits_{j} b_j\left|j\right\rangle$



# Quantum Memories (aka qRAM)

**Problem:** It is not clear if we will be able to do this eﬃciently at all, let alone in a fault-tolerant setting. 

 ❗ _An algorithmic speedup **may not** translate to an actual speedup in an application if it is not eﬃcient to use data in the ﬁrst place!_

😓 Physical limitations like coherence time, error rates, hardware supported gates, etc. contribute to the difficulty.

💡 There are many different approaches, each optimizing for a particular resource. 


## Deep Dive: qRAM approaches and tradeoffs:

#### http://bit.ly/between-the-bitlines

<center>
    <img src="media/olivia-talk-title.png" width="48%">

</center>

### So what's the path forward?

- To find out **if qRAM will ever work**, we need to have a good way to **evaluate different proposals**.

- How can we compare tons of papers when none provide implementations?

<center>
    <img src="https://media.giphy.com/media/lQ6iahDJqm9oldX5gh/source.gif" width="40%">
</center> 

### Transparency is _critical_ for resource estimations

- Assumptions about our testing and implementations need to be made uniformly and clearly
- Others need to be able to reproduce our results and potentially modify to meet assumptions in their work

<center>
    <img src="https://media.giphy.com/media/c5eqVJN7oNLTq/source.gif" width="40%">
</center>

## We need:

- A way to implement many different algorithms representing qRAM proposals,
- count the resources needed for each proposal, and
- do it in an open and reproducible way.

<center>
    <img src="media/wanted.png" width="25%">
</center>


### https://github.com/qsharp-community/qram


<center>
    <img src="media/github-screencap.png" width="50%">
</center>

## Basic layout:

```
├───📃 docs 📃
├───🔮 samples 🔮
│   ├───BucketBrigade
│   ├───Grover
│   ├───Qrom
│   ├───ResourceEstimation
│   └───SelectSwap
├───✨ src ✨
└───🧪 tests 🧪
```

# `src`: where qRAMs are implemented
<center>
    <img src="media/src-screenshot.png" width="60%">
</center>

### Currently implemented proposals:
**qRAM**
- Bucket Brigade (bit and phase queries) [0708.1879](https://arxiv.org/abs/0708.1879)

**qROM**
- Simple [0807.4994](https://arxiv.org/abs/0807.4994)
- SELECTSWAP [1812.00954](https://arxiv.org/abs/1812.00954)

## Using a qROM

In [ ]:
%package QSharpCommunity.Libraries.Qram::0.1.35

## Libraries help us bootstrap

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;
open Qram;

## Custom Types for quantum memories

```
newtype QRAM = (
    QueryPhase : ((AddressRegister, MemoryRegister, Qubit[]) => Unit is Adj + Ctl),
    QueryBit : ((AddressRegister, MemoryRegister, Qubit[]) => Unit is Adj + Ctl), 
    Write : ((MemoryRegister, MemoryCell) => Unit), 
    AddressSize : Int,
    DataSize : Int
);
```

## We can use function and custom types to handle classical data

In [ ]:
function ExampleMemoryData() : MemoryBank {
    let numDataBits = 3;
    let data =  [
        (0, IntAsBoolArray(0, numDataBits)), 
        (2, IntAsBoolArray(5, numDataBits)),
        (4, IntAsBoolArray(2, numDataBits)),
        (5, IntAsBoolArray(3, numDataBits))
    ];
    return GeneratedMemoryBank(Mapped(MemoryCell, data));
}

### Using a qROM (Read Only Memory)

In [ ]:
operation QueryAndMeasureQrom(memory : QROM, queryAddress : Int) : Int {
    using ((addressRegister, targetRegister) = (Qubit[memory::AddressSize], Qubit[memory::DataSize])) {
        // Encode our query address
        ApplyPauliFromBitString (PauliX, true, IntAsBoolArray(queryAddress, memory::AddressSize), addressRegister);
        
        // Read from the memory
        memory::Read(LittleEndian(addressRegister), targetRegister);
        ResetAll(addressRegister);
        
        // Measure the target register as an integer
        return MeasureInteger(LittleEndian(targetRegister));
    }
}

In [ ]:
operation QueryQrom(queryAddress : Int) : Int {
    // Generate a (Int, Bool[]) array of data.
    let data = ExampleMemoryData();
    // Create the QRAM.
    let memory = QromOracle(data::DataSet);
    // Measure and return the data value stored at `queryAddress`.
    return QueryAndMeasureQrom(memory, queryAddress);
}

### Using a qROM

```
// data: {(0, 0), (2, 5), (4, 2), (5, 3)}
```

In [ ]:
%simulate QueryQrom queryAddress=2

In [ ]:
%estimate QueryQrom queryAddress=2

# `tests`: ✔ our work
- Can run small instances on simulators
- Can verify resource counts on larger instances

<center>
    <img src="media/tests-screenshot.png" width="40%">
</center>


# `docs`: help others use our work💪

<center>
    <img src="media/docs.png" width="60%">
</center>

# What's next for the qRAM library?

🔍 Detailed resource counting for subroutines of our programs

📓 More documentation in an interactive browser

📄 Research paper compiling our results

❓ More qRAM/qROM proposals 

<figure style="text-align: left;">
        <caption>
    </caption>
    <img src="media/milestones.png" width="50%">

</figure>

# Part 4: How can you start your own Q# project?

### a.k.a. best pratice open source devlopment for research!

### 💾 Open Source Software (OSS)
Software that can be freely accessed, used, changed, and shared (in modified or unmodified form) by anyone.

### 🙌Open Source Community
Everything _except_ the code

- Licences
- Processes and governance
- Funding
- Blogging and Social Media
- Diversity
- etc.

## Let's `import` OSS best pratices to the quantum relm!
There are _**tons**_ of resources for building both open source projects and communities from the classical software community:
- https://opensourcediversity.org/
- https://opensource.guide/building-community/
- Talk: [Building Open Source Communities - Tierney Cyren](https://www.youtube.com/watch?v=9owWEY5pmJg)

<!--<figure style="margin-left:auto; margin-right:auto;">
    <img src="https://media.giphy.com/media/iHyVaHfEYXZos8qPX2/source.gif" width="35%">
    <caption>
      <br>  
        <strong></strong>
    </caption>
</figure>-->




## What does open source research look like?

- All source code/data is included when publishing
    - Raw data is included if possible, as well as all suplimentary instructions you used to get from that data to your results
    - Automation (and OSS) is your friend 😁
- Cite and donate to open source projects used in your research
- Use free open source tools and platforms so it is easy for others to use and extend your work

## We need to find (or create!) OSS communities in Quantum Computing

- QOSF: Quantum Open Source Foundation
- Q# Community
- WIQCA: Women in Quantum Computing and Applications
- Unitary Fund
- ...and more!

❗ We also need to understand how the _quantum research_ and _industry_ communities influence our communities.

## Q# community: a usergroup suporting open source quantum development + research

**Mission:** we want to _empower everyone_ to get involved in quantum development.

- Make sure everyone feels **safe and welcome** in our spaces


- Understand **how the community communicates**, and setup tools that work for them $\to$ [bit.ly/qsharp-slack](bit.ly/qsharp-slack) 

- Ensure we can support for **members of all skill levels**

## Check us out @ [qsharp.community](qsharp.community)!

<center>
    <img src="media/qsc.png" width="50%">
</center>

## Selection of other Q# community projects:

<figure style="text-align: left;">
    <img src="media/qsc-projects.png" width="80%">
</figure>

## Case study: How does the qRAM team work?

- Standing time each day for core devs to check in if needed (sort out blocking issues)
- Slack channel for design discussions and issue/bug feedback
- We use multiple operating systems and development toolsets, so tools like Visual Studio Code was great
- Program with the community in the open, with everyone. Literally! 
    - [Twitch.tv/crazy4pi314](https://www.twitch.tv/crazy4pi314) + Visual Studio/Visual Studio Code [Liveshare](https://visualstudio.microsoft.com/services/live-share/)
    <figure style="text-align: left;">
    <img src="https://visualstudio.microsoft.com/wp-content/uploads/2018/11/v2-Edit-Comp_FINAL-optimized840.gif" width="60%">
</figure>

## New 🔨 we made while building qRAM

- Q# library [project templates](https://github.com/crazy4pi314/qsharp-library-template) to make it easier for others to start building a new library for Q# from scratch
    - [Contribution guide](https://github.com/qsharp-community/qram/blob/master/CONTRIBUTING.md)
    - [Code of Conduct](https://github.com/qsharp-community/qram/blob/master/CODE_OF_CONDUCT.md)
    - [Basic build automation + testing framework](https://github.com/qsharp-community/qram/actions)
- Containers/portable development environments so that we don't have the "well it worked on my machine" problem
    - Tools like [MyBinder](https://mybinder.org/) and [remote development environments](https://code.visualstudio.com/docs/remote/containers) can be super helpful

## Want to start doing research in Q#?

📑 [docs.microsoft.com/quantum](docs.microsoft.com/quantum)

📗 _Learn Quantum Computing with Python and Q#_: [bit.ly/qsharp-book](bit.ly/qsharp-book)

📺 Live quantum development on Twitch: [twitch.tv/crazy4pi314](twitch.tv/crazy4pi314)

🧩 Template repo for Q# projects complete with LaTeX paper setup [TODO]()


# 📝 Review 📝

- Q# is a high level, open-source language for writing _quantum algorithms_ in a portable and reproducable way ✔
- We have implemented a number of proposals for qRAMs in a Q# library and are working on resource estimation ✔
- The open source community is a great resource for reproducable research ✔
    - Q# community is a great place for help when you are working on Q# projects ✔

## 👩‍💻Quantum programming resources!👩‍💻

- Q# Documentation: [docs.microsoft.com/quantum](https://docs.microsoft.com/quantum)
- _Learn Quantum Computing with Python and Q\#_ : [bit.ly/qsharp-book](http://www.manning.com/?a_aid=learn-qc-kaiser)
- Community: [qsharp.community](https://qsharp.community/)
    - Q# Community Slack [bit.ly/qsharp-slack](https://bit.ly/qsharp-slack)
    - Women in Quantum Computing and Algorithms (WIQCA): [wiqca.dev](https://wiqca.dev)
    - Quantum Open Source Foundation: [qosf.org](https://www.qosf.org/)
    - Unitary Fund [unitary.fund](https://unitary.fund/)